In [37]:
import os
import sys
from math import ceil, floor

In [26]:
# list files in the results directory
folder = "nc"
files = os.listdir(folder)
files

['cbsstudy_individualDelays_il_10_s_1.txt',
 'cbsstudy_individualDelays_il_10_s_10.txt',
 'cbsstudy_individualDelays_il_10_s_10_withCT.txt',
 'cbsstudy_individualDelays_il_10_s_1_withCT.txt',
 'cbsstudy_individualDelays_il_10_s_2.txt',
 'cbsstudy_individualDelays_il_10_s_2_withCT.txt',
 'cbsstudy_individualDelays_il_10_s_3.txt',
 'cbsstudy_individualDelays_il_10_s_3_withCT.txt',
 'cbsstudy_individualDelays_il_10_s_4.txt',
 'cbsstudy_individualDelays_il_10_s_4_withCT.txt',
 'cbsstudy_individualDelays_il_10_s_5.txt',
 'cbsstudy_individualDelays_il_10_s_5_withCT.txt',
 'cbsstudy_individualDelays_il_10_s_6.txt',
 'cbsstudy_individualDelays_il_10_s_6_withCT.txt',
 'cbsstudy_individualDelays_il_10_s_7.txt',
 'cbsstudy_individualDelays_il_10_s_7_withCT.txt',
 'cbsstudy_individualDelays_il_10_s_8.txt',
 'cbsstudy_individualDelays_il_10_s_8_withCT.txt',
 'cbsstudy_individualDelays_il_10_s_9.txt',
 'cbsstudy_individualDelays_il_10_s_9_withCT.txt',
 'cbsstudy_individualDelays_il_11_s_1.txt',
 'cb

# Read DYRECTsn Output from File and parse Idle Slopes

In [27]:
ncStageIdleSlopes = dict()
ncAggregateIdleSlopes = dict()
for file in files:
    # read file from the back to the front
    ctScheme = "NoPCTRes"
    if "withCT" in file:
        ctScheme = "PCTRes"
    if ctScheme not in ncStageIdleSlopes:
        ncStageIdleSlopes[ctScheme] = dict()
        ncAggregateIdleSlopes[ctScheme] = dict()
    il = int(file.split("il_")[1].split("_")[0])
    s = int(file.split("_s_")[1].split("_")[0].split(".")[0])
    if (il, s) not in ncStageIdleSlopes[ctScheme]:
        ncStageIdleSlopes[ctScheme][(il, s)] = dict()
    with open(folder + "/" + file, "r") as f:
        lines = f.readlines()
        # read lines in reverse
        for line in lines[::-1]:
            if "Successfully" in line:
                split = line.strip().split(":")[-1].split("of")
                if split[0].strip() != split[1].strip():
                    print(f"Error: reservation unsuccessful for {file}")
            elif "Hop" in line:
                break ## reading reservation done
            elif "bit/s" in line:
                idleslope = int(ceil(float(line.split(":")[1].strip())))
                link = line.split(")")[0].replace("(","").strip().split(",")
                if "aggregate" in link[0]:
                    ncAggregateIdleSlopes[ctScheme][(il, s)] = idleslope
                    break 
                else:
                    fromSwitch = int(link[0].split("stage_")[1])
                    ncStageIdleSlopes[ctScheme][(il, s)][fromSwitch] = idleslope
# sort the results
print(ncStageIdleSlopes)
print(ncAggregateIdleSlopes)

{'NoPCTRes': {(10, 1): {0: 16717292}, (10, 10): {9: 60074568, 8: 57623294, 7: 54851333, 6: 51691349, 5: 48055737, 4: 43828374, 3: 38851984, 2: 32908143, 1: 25684353, 0: 16717292}, (10, 2): {1: 25684353, 0: 16717292}, (10, 3): {2: 32908143, 1: 25684353, 0: 16717292}, (10, 4): {3: 38851984, 2: 32908143, 1: 25684353, 0: 16717292}, (10, 5): {4: 43828374, 3: 38851984, 2: 32908143, 1: 25684353, 0: 16717292}, (10, 6): {5: 48055737, 4: 43828374, 3: 38851984, 2: 32908143, 1: 25684353, 0: 16717292}, (10, 7): {6: 51691349, 5: 48055737, 4: 43828374, 3: 38851984, 2: 32908143, 1: 25684353, 0: 16717292}, (10, 8): {7: 54851333, 6: 51691349, 5: 48055737, 4: 43828374, 3: 38851984, 2: 32908143, 1: 25684353, 0: 16717292}, (10, 9): {8: 57623294, 7: 54851333, 6: 51691349, 5: 48055737, 4: 43828374, 3: 38851984, 2: 32908143, 1: 25684353, 0: 16717292}, (11, 1): {0: 15248594}, (11, 10): {9: 57378302, 8: 54886549, 7: 52085359, 6: 48913275, 5: 45291414, 4: 41116812, 3: 36252483, 2: 30512102, 1: 23635586, 0: 15248

In [28]:
# try to find similarities in the slopes
for ctScheme in ncStageIdleSlopes:
    for fromSwitch in ncStageIdleSlopes[ctScheme][(13,10)]:
        # check for all other il and s combination whether the fromSwitch has the same slope
        idleslope = ncStageIdleSlopes[ctScheme][(13,10)][fromSwitch]
        for il in range(2, 13):
            for s in range(1, 10):
                if fromSwitch >= s:
                    continue
                elif ncStageIdleSlopes[ctScheme][(il, s)][fromSwitch] != idleslope:
                    # print(f"Error: {ctScheme} {fromSwitch} {il} {s} {ncStageIdleSlopes[ctScheme][(il, s)][fromSwitch]} != {idleslope}")
                    pass
                else:
                    print(f"{ctScheme} {fromSwitch} {il} {s} {ncStageIdleSlopes[ctScheme][(il, s)][fromSwitch]} == {idleslope}")


# Export to OMNET.ini Study

In [76]:
def writePCTGen(f, il, s, stageIdleSlopes):
    sToUs = 1000000
    for fromSwitch in stageIdleSlopes:
        prefix =f"**.ctSrc[{fromSwitch}].services[0].startTime = 0.1s - 1us + (500ns*ancestorIndex(2)) + ({fromSwitch}*8us) "
        totalBandwidth = 75000000
        linkrate = 100000000
        talkerInterval = 0.000125
        ctInterval = 0.1
        talkerSize = floor(((totalBandwidth / 8.0) * talkerInterval) / il)
        ctSize = floor(((totalBandwidth / 8.0) * talkerInterval) - (((totalBandwidth / 8.0) * talkerInterval) / il))
        delayAsStr = prefix
        for switch in range(0, fromSwitch):
            idleslope = stageIdleSlopes[switch]
            sendslope = linkrate - idleslope
            delay = ((ctSize * 8.0) * (sendslope/linkrate)) / idleslope
            delayAsStr += " + " + format(delay*sToUs, ".3f") + "us"
        transmissionDelay = ((talkerSize-12) * 8 / linkrate)
        ctDelay = ((ctSize-12) * 8 / linkrate)
        delayAsStr += f" + ({fromSwitch+1})*" + format(transmissionDelay*sToUs, ".3f") + "us"
        delayAsStr += f" + ({fromSwitch-1})*" + format(ctDelay*sToUs, ".3f") + "us"
        f.write(delayAsStr + "\n")

def writePCTGenAggregate(f, il, s, stageIdleSlopes):
    sToUs = 1000000
    prefix =f"**.aggregateCTSrc.services[*].startTime = 0.1s - 1us + ({s}*8us)"
    totalBandwidth = 75000000
    linkrate = 100000000
    talkerInterval = 0.000125
    ctInterval = 0.1
    talkerSize = floor(((totalBandwidth / 8.0) * talkerInterval) / il)
    ctSize = floor(((totalBandwidth / 8.0) * talkerInterval) - (((totalBandwidth / 8.0) * talkerInterval) / il))
    delayAsStr = prefix
    for switch in stageIdleSlopes:
        idleslope = stageIdleSlopes[switch]
        sendslope = linkrate - idleslope
        delay = ((ctSize * 8.0) * (sendslope/linkrate)) / idleslope
        delayAsStr += " + " + format(delay*sToUs, ".3f") + "us"
    transmissionDelay = ((talkerSize-12) * 8 / linkrate)
    ctDelay = ((ctSize-12) * 8 / linkrate)
    delayAsStr += f" + ({s+1})*" + format(transmissionDelay*sToUs, ".3f") + "us"
    delayAsStr += f" + ({s})*" + format(ctDelay*sToUs, ".3f") + "us"
    delayAsStr += " - second(((1522)*8)/100000000)"
    f.write(delayAsStr + "\n")

In [77]:

def setIdleSlopeStageSwitch(stage, maxstage, idleslope):
    return f"**.stageSwitch[{stage}].etherMAC[1].shaper.transmissionSelectionAlgorithm[7].staticIdleSlope = {idleslope}bps"

def setAggregateSwitch(idleslope):
    return f"**.aggregateSwitch.etherMAC[0].shaper.transmissionSelectionAlgorithm[7].staticIdleSlope = {idleslope}bps"

def writeConfig(ctScheme, il, s, stageIdleSlopes, aggregateIdleSlope, f):
    config = f"[Config NC_{ctScheme}_IL-{il}_S-{s}]\n"
    f.write(config)
    f.write("extends=NC\n")    
    # f.write("**.numInputLinks = ${IL=" + str(il) + "}\n")
    # f.write("**.numStages = ${S=" + str(s) + "}\n")
    for fromSwitch in ncStageIdleSlopes[ctScheme][(il, s)]:
        idleslope = stageIdleSlopes[fromSwitch]
        f.write(setIdleSlopeStageSwitch(fromSwitch, s-1, idleslope) + "\n")
    f.write(setAggregateSwitch(aggregateIdleSlope) + "\n")
    f.write("\n")

def writeNCPrioCTConfig(il, s, stageIdleSlopes, f):
    ctScheme = "PCTRes"
    config = f"[Config NC_PCT_IL-{il}_S-{s}]\n"
    f.write(config)
    f.write("extends=CrossTraffic\n")
    f.write("# register be ct but never actually send a message, stop when subscriber registers\n")
    f.write("**.ctSrc[*].services[2].stopTime = 0.12s\n")
    writePCTGen(f, il, s, stageIdleSlopes)
    writePCTGenAggregate(f, il, s, stageIdleSlopes)
    f.write("\n")

def writeStudies(f, il, s):
    f.write(f"[Config Study_NC_NoCT_IL-{il}_S-{s}]\n")
    f.write(f"extends=NC_PCTRes_IL-{il}_S-{s}\n")
    f.write("\n")
    f.write(f"[Config Study_NC_BECT_IL-{il}_S-{s}]\n")
    f.write(f"extends=NC_PCTRes_IL-{il}_S-{s},BECrossTraffic\n")
    f.write("\n")
    f.write(f"[Config Study_NC_PCT_IL-{il}_S-{s}]\n")
    f.write(f"extends=NC_PCTRes_IL-{il}_S-{s},NC_PCT_IL-{il}_S-{s}\n")
    f.write("\n")
    f.write(f"[Config Study_NC_NoCT_NoPCTRes_IL-{il}_S-{s}]\n")
    f.write(f"extends=NC_NoPCTRes_IL-{il}_S-{s}\n")
    f.write("\n")
    f.write(f"[Config Study_NC_BECT_NoPCTRes_IL-{il}_S-{s}]\n")
    f.write(f"extends=NC_NoPCTRes_IL-{il}_S-{s},BECrossTraffic\n")
    f.write("\n")
    

In [78]:
outfile = "../nc_config.ini"
with open(outfile, "w") as f:
    f.write("# This file is generated by the script\n")
    f.write("# All studies are to be executed only for the IL and S combination in their name.\n")
    f.write("[Config NC] ## defined in omnetpp.ini\n")
    f.write("\n")
    for ctScheme in ncStageIdleSlopes:
        for key in ncStageIdleSlopes[ctScheme]:
            il = key[0]
            s = key[1]
            writeConfig(ctScheme, il, s, ncStageIdleSlopes[ctScheme][key], ncAggregateIdleSlopes[ctScheme][key], f)
            if ctScheme == "PCTRes":
                writeNCPrioCTConfig(il, s, ncStageIdleSlopes[ctScheme][key], f)
    f.write("#################################################\n#################### STUDIES ####################\n#################################################\n")
    for ctScheme in ncStageIdleSlopes:
        for key in ncStageIdleSlopes[ctScheme]:
            il = key[0]
            s = key[1]
            writeStudies(f, il, s)
        break